### Trích xuất kết quả
Trích xuất các meta data của kết quả của lần đánh giá vào một file json bao gồm:
| **Trường**                   | **Ý nghĩa**                                                                 |
|---------------------------|------------------------------------------------------------------------|
| eval_id                | Id của trường hợp đang được đánh giá (định danh duy nhất cho mỗi record). |
| session_id             | Id của phiên hội thoại (chat session) tương ứng.                        |
| tool_trajectory_avg_score | Điểm trung bình của quỹ đạo gọi tool (độ chính xác và hợp lý khi sử dụng công cụ). |
| response_match_score   | Điểm chất lượng phản hồi, đo bằng độ tương đồng ROUGE với phản hồi chuẩn. |

In [3]:
import os
import json

folder_path = "../book_review_agent/.adk/parsed"
eval_set = "focus_unclear"

eval_info = []

for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)
            eval_id = eval_set + data["eval_case_results"][0]["eval_id"]
            session_id = data["eval_case_results"][0]["session_id"]

            tool_trajectory_avg_score = 0
            response_match_score = 0

            overall = data["eval_case_results"][0]["overall_eval_metric_results"]

            for metric in overall:
                if metric["metric_name"] == "tool_trajectory_avg_score":
                    tool_trajectory_avg_score = metric["score"]
                elif metric["metric_name"] == "response_match_score":
                    response_match_score = metric["score"]

            eval_info.append(
                {
                    "eval_id": eval_id,
                    "session_id": session_id,
                    "tool_trajectory_avg_score": tool_trajectory_avg_score,
                    "response_match_score": response_match_score,
                }
            )

output_filepath = f"data/{eval_set}_eval.json"

with open(output_filepath, "w", encoding="utf-8") as output_file:
    json.dump(eval_info, output_file, indent=2, ensure_ascii=False)

print(f"Dữ liệu đã được lưu vào {output_filepath}")

Dữ liệu đã được lưu vào data/focus_unclear_eval.json


### Lưu kết quả cụ thể
Lưu kết quả cụ thể của các phiên chạy vào folder `../session_traces` dựa vào file kết quả vừa trích xuất.

In [4]:
import os
import json
import requests

# Đường dẫn đến tệp JSON chứa danh sách session_id
input_filepath = f'data/{eval_set}_eval.json'

# Thư mục để lưu các tệp session trace
output_folder = f'session_traces_{eval_set}'
os.makedirs(output_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

# URL endpoint
base_url = 'http://127.0.0.1:8000/debug/trace/session'

# Đọc danh sách session_id từ tệp JSON
with open(input_filepath, 'r', encoding='utf-8') as file:
    eval_info = json.load(file)

for item in eval_info:
    session_id = item["session_id"]
    url = f"{base_url}/{session_id}"
    print(f"Fetching session trace for session_id: {session_id}")

    try:
        # Gửi yêu cầu GET
        response = requests.get(url)
        response.raise_for_status()  # Kiểm tra nếu có lỗi HTTP

        # Lưu kết quả vào tệp JSON
        output_filepath = os.path.join(output_folder, f"{session_id}.json")
        with open(output_filepath, 'w', encoding='utf-8') as output_file:
            json.dump(response.json(), output_file, indent=4, ensure_ascii=False)
        
        print(f"Saved session trace to: {output_filepath}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch session trace for session_id {session_id}: {e}")

Fetching session trace for session_id: ___eval___session___357481c2-cccc-4e9a-9cb6-bbfa97fbc2d8
Saved session trace to: session_traces_focus_unclear/___eval___session___357481c2-cccc-4e9a-9cb6-bbfa97fbc2d8.json
Fetching session trace for session_id: ___eval___session___a4976f2b-4cb8-41c7-a837-308282b0cb1c
Saved session trace to: session_traces_focus_unclear/___eval___session___a4976f2b-4cb8-41c7-a837-308282b0cb1c.json
Fetching session trace for session_id: ___eval___session___0717b109-3e64-4a6a-92b3-49e87fd93a2c
Saved session trace to: session_traces_focus_unclear/___eval___session___0717b109-3e64-4a6a-92b3-49e87fd93a2c.json
Fetching session trace for session_id: ___eval___session___f7c3646d-e66e-4b86-a857-a5fb0cfbc39d
Saved session trace to: session_traces_focus_unclear/___eval___session___f7c3646d-e66e-4b86-a857-a5fb0cfbc39d.json
Fetching session trace for session_id: ___eval___session___763d914c-4005-4ec1-8163-44e9d4ccb64e
Saved session trace to: session_traces_focus_unclear/___eval

### Trích xuất dữ liệu chi tiết từ các file đã lưu

File kết quả gồm 100 bản ghi, các trường dữ liệu được mô tả như sau:

| **Trường**                   | **Ý nghĩa**                                                                 |
|---------------------------|------------------------------------------------------------------------|
| eval_id                | Id của trường hợp đang được đánh giá (định danh duy nhất cho mỗi record). |
| session_id             | Id của phiên hội thoại (chat session) tương ứng.                        |
| tool_trajectory_avg_score | Điểm trung bình của quỹ đạo gọi tool (độ chính xác và hợp lý khi sử dụng công cụ). |
| response_match_score   | Điểm chất lượng phản hồi, đo bằng độ tương đồng ROUGE với phản hồi chuẩn. |
| invocation_duration_ms | Thời gian thực thi toàn bộ (bao gồm model, tool và overhead), tính bằng millisecond. |
| llm_duration_ms        | Thời gian mà LLM tiêu tốn để sinh phản hồi.                             |
| tool_duration_ms       | Tổng thời gian các công cụ được gọi chạy thực tế.                        |
| overhead_duration_ms   | Thời gian overhead phát sinh ngoài LLM và tool (ví dụ chuẩn bị dữ liệu, xử lý trung gian). |
| total_input_tokens     | Tổng số tokens đầu vào mà LLM nhận trong toàn bộ phiên.                  |
| total_output_tokens    | Tổng số tokens đầu ra mà LLM sinh ra trong toàn bộ phiên.                |
| total_tokens           | Tổng số tokens đã dùng (đầu vào + đầu ra).                              |
| num_llm_calls          | Số lần LLM được gọi trong một bản ghi.                                  |
| num_tool_calls         | Số lần công cụ được gọi.                                                |
| total_spans            | Tổng số đoạn (spans) được theo dõi/trích xuất trong toàn bộ quá trình.   |


In [9]:
import os
import json
import requests
from tqdm import tqdm

# --- 1. Cấu hình Toàn cục ---
# Tệp JSON chứa danh sách các case cần đánh giá
INPUT_EVAL_FILE = f"data/{eval_set}_eval.json"
# Thư mục để lưu trữ (cache) các file trace đã tải về
SESSION_TRACES_FOLDER = f"session_traces/{eval_set}"
# Tệp JSON đầu ra cuối cùng chứa kết quả phân tích
FINAL_OUTPUT_FILE = f"data/{eval_set}_eval_detail.json"

# URL endpoint để lấy trace
BASE_URL = "http://127.0.0.1:8000/debug/trace/session"

# Danh sách các agent cha mục tiêu để lọc các lần gọi LLM
TARGET_PARENT_AGENTS = {
    "agent_run [creative_assistant_agent]",
    "agent_run [writer_agent]",
    "agent_run [critic_agent]",
    "agent_run [save_draft_agent]",
    "agent_run [confirmation_agent]",
    "agent_run [research_agent]",
}

# --- 2. Logic Chính ---

# Tạo thư mục lưu trữ nếu chưa có
os.makedirs(SESSION_TRACES_FOLDER, exist_ok=True)

# Đọc tệp JSON đầu vào
if not os.path.exists(INPUT_EVAL_FILE):
    print(f"Lỗi: Không tìm thấy tệp đầu vào tại '{INPUT_EVAL_FILE}'")
else:
    with open(INPUT_EVAL_FILE, "r", encoding="utf-8") as file:
        eval_info = json.load(file)

    # Lặp qua từng case đánh giá với thanh tiến trình
    for item in tqdm(eval_info, desc="Processing and Analyzing Sessions"):
        session_id = item.get("session_id")
        if not session_id:
            continue

        trace_data = None
        trace_filepath = os.path.join(SESSION_TRACES_FOLDER, f"{session_id}.json")

        try:
            # --- Bước A: Lấy dữ liệu Trace (từ file cache hoặc API) ---
            if os.path.exists(trace_filepath):
                # Nếu đã có file, đọc từ cache
                with open(trace_filepath, "r", encoding="utf-8") as trace_file:
                    trace_data = json.load(trace_file)
            else:
                # Nếu chưa có, gọi API để tải về
                response = requests.get(url=f"{BASE_URL}/{session_id}")
                response.raise_for_status()
                trace_data = response.json()
                # Và lưu lại vào cache cho lần sau
                with open(trace_filepath, "w", encoding="utf-8") as output_file:
                    json.dump(trace_data, output_file, indent=4, ensure_ascii=False)

            # --- Bước B: Phân tích dữ liệu Trace đã có ---
            if trace_data and isinstance(trace_data, list):
                # Khởi tạo các biến
                total_input_tokens, total_output_tokens = 0, 0
                llm_duration_ns, tool_duration_ns = 0, 0
                num_llm_calls, num_tool_calls = 0, 0

                span_id_to_name_map = {
                    span["span_id"]: span.get("name") for span in trace_data
                }

                for trace in trace_data:
                    duration_ns = trace.get("end_time", 0) - trace.get("start_time", 0)

                    if trace.get("name") == "call_llm":
                        parent_id = trace.get("parent_span_id")
                        if (
                            parent_id
                            and span_id_to_name_map.get(parent_id)
                            in TARGET_PARENT_AGENTS
                        ):
                            num_llm_calls += 1
                            llm_duration_ns += duration_ns
                            attrs = trace.get("attributes", {})
                            total_input_tokens += attrs.get(
                                "gen_ai.usage.input_tokens", 0
                            )
                            total_output_tokens += attrs.get(
                                "gen_ai.usage.output_tokens", 0
                            )

                    elif str(trace.get("name")).startswith("execute_tool"):
                        num_tool_calls += 1
                        tool_duration_ns += duration_ns

                root_span = next(
                    (s for s in trace_data if not s.get("parent_span_id")),
                    trace_data[-1],
                )
                invocation_duration_ns = root_span.get("end_time", 0) - root_span.get(
                    "start_time", 0
                )

                # Thêm tất cả các trường kết quả vào item
                item["invocation_duration_ms"] = invocation_duration_ns / 1_000_000
                item["llm_duration_ms"] = llm_duration_ns / 1_000_000
                item["tool_duration_ms"] = tool_duration_ns / 1_000_000
                item["overhead_duration_ms"] = (
                    invocation_duration_ns - llm_duration_ns - tool_duration_ns
                ) / 1_000_000
                item["total_input_tokens"] = total_input_tokens
                item["total_output_tokens"] = total_output_tokens
                item["total_tokens"] = total_input_tokens + total_output_tokens
                item["num_llm_calls"] = num_llm_calls
                item["num_tool_calls"] = num_tool_calls
                item["total_spans"] = len(trace_data)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"\nLỗi xử lý session {session_id}: {e}")
            # Gắn None cho các trường nếu có lỗi
            keys_to_nullify = [
                "invocation_duration_ms",
                "llm_duration_ms",
                "tool_duration_ms",
                "overhead_duration_ms",
                "total_input_tokens",
                "total_output_tokens",
                "total_tokens",
                "num_llm_calls",
                "num_tool_calls",
                "total_spans",
            ]
            for key in keys_to_nullify:
                item[key] = None

    # --- 3. Lưu kết quả cuối cùng ---
    with open(FINAL_OUTPUT_FILE, "w", encoding="utf-8") as output_file:
        json.dump(eval_info, output_file, indent=4, ensure_ascii=False)

    print(
        f"\nQuy trình hoàn tất! Dữ liệu đã được làm giàu và lưu vào {FINAL_OUTPUT_FILE}"
    )

Processing and Analyzing Sessions: 100%|██████████| 100/100 [00:00<00:00, 112.80it/s]


Quy trình hoàn tất! Dữ liệu đã được làm giàu và lưu vào data/focus_unclear_eval_detail.json


### Lưu dữ liệu thành csv

In [8]:
import pandas as pd

# Tên tệp JSON đầu vào
input_filename = f'data/{eval_set}_eval_detail.json' # Hãy chắc chắn rằng bạn đã lưu file JSON với tên này

# Tên tệp CSV đầu ra
output_filename = f'data/{eval_set}_eval_detail.csv'

# Đọc trực tiếp từ tệp JSON vào DataFrame
df = pd.read_json(input_filename)

# Ghi DataFrame ra tệp CSV
df.to_csv(output_filename, index=False, encoding='utf-8')

print(f"Chuyển đổi thành công! Dữ liệu đã được lưu vào tệp '{output_filename}'")

Chuyển đổi thành công! Dữ liệu đã được lưu vào tệp 'data/focus_unclear_eval_detail.csv'
